## Analysis of Trove Government Gazettes

Attempting to reproduce the work described [on the NLA blog](https://www.nla.gov.au/blogs/trove/2018/07/23/digital-tools-for-big-research) where a collection of Certificates of Naturalisation were selected from the Trove Government Gazettes and analysed to give a picture of the number of arrivals over time. 

In that exercise the work was done manually to identify names and generate counts.  I will attempt to implement an automated process to derive the same data. 

This notebook makes use of a number of functions defined in the `gazette` module in this directory.  These are not 
included in the notebook to make it easier to read but the code is accessible in that file
for reference. 

In [ ]:
# install all requirements quietly
!pip install -q -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import spacy

# import two local modules
import utils
import gazette

## Load Spacy 

[Spacy](https://spacy.io/) is a Python module for Natural Language Processing. It is capable of doing Named Entity Recognition but our experiments on the target texts show that it doesn't perform very well because there is little context in the lists of names to provide the usual cues to whether a word is a name or not.  

However, Spacy provides some useful pattern matching functionality that we can use to find names within this text. So we load a model here and create the Spacy NLP processor.

In [ ]:
# download the spacy models we need
SPACY_MODEL = 'en_core_web_sm'
#spacy.cli.download(SPACY_MODEL)
nlp = spacy.load(SPACY_MODEL)

We use the function defined in the `utils` module to query the Trove API for "Certificates of Naturalisation" - these are the articles in the Government Gazzettes that contain lists of names of recently naturalised citizens.  Our goal is to extract from these the details of the names and addresses of these people.   We ask for 1000 matches to the query.

Note that the query procedure will save the results in a cache file so that we don't query Trove
too many times for large numbers of documents (it's also much faster to load a file than query the API).

In [ ]:
articles = utils.trove_query_cached('"Certificates of Naturalisation"', 1000, cachefile="articles.json")
articles.head()

In [ ]:
## look at the text of the first article. The text is marked up in HTML with one <span> element
## per line. 

articles.iloc[0].articleText

The next stage uses the Spacy NLP module to match names, addresses and dates within the text.   The procedure is defined in the functions in the `gazette` module, look there for the detail but I will outline it here.

First we find the text of the article. This is done by removing the HTML tags from the text but we also skip 
the inital lines of the text that are the header of the article.  We do this by ignoring lines up to
one containg the word 'Secretary' since that is generally the last line before the list of names begins.

The text is added as a column (`text`) in the data frame.

In [ ]:
articles['text'] = articles.apply(gazette.trove_naturalisation_text, axis=1)
articles.head()

In [ ]:
## let's look at the stripped text for the first article
print(articles.iloc[0].text)

The text is structured as a series of records

> Last, First, Address, Date

the approach to finding these is to identify the dates via pattern matching, then split the text on the comma character to extract the name and address parts.   

Spacy has a powerful pattern matcher that let's us write flexible patterns to match dates:

```python
matcher.add('DATE', None,
            [{'SHAPE': 'd.d.dd'}],
            [{'SHAPE': 'd,d.dd'}],
            [{'SHAPE': 'd.dd.dd'}],
            [{'SHAPE': 'dd.d.dd'}],
            [{'SHAPE': 'dd.dd.dd'}],
            [{'SHAPE': 'dd.d.dddd'}],
            [{'SHAPE': 'dd.dd.dddd'}],
            [{'SHAPE': 'd.d.dddd'}],
            [{'SHAPE': 'd.dd.dddd'}],
            [{'SHAPE': 'dd'}, {'IS_SPACE': True}, {'SHAPE': 'd.dd'}],
            [{'SHAPE': 'dd'}, {'IS_SPACE': True}, {'SHAPE': 'dd.dd'}],
            [{'IS_DIGIT': True}, {'IS_SPACE': True}, {'IS_DIGIT': True}, {'IS_SPACE': True}, {'IS_DIGIT': True}]
            )
```

This pattern matches dates like '15.12.66' and '15 12 66' and a few other variations that are observed in the text.  

Once dates have been identified we can select the text between dates and split it at each comma to identify 
the name and address.  The result is a set of __tags__ for each text showing the start and end of each 
entity.   

We apply this procedure to every article in the dataframe and add the result as a new column `tags`.  Note
that this step takes some time (about five minutes on my laptop) since we are doing quite a bit of work on 1000 documents.

In [ ]:
articles['tags'] = articles.apply(gazette.tag_row, axis=1, args=(nlp,))

In [ ]:
## look at the tags for the first article (sorted to put them in order of the start offset)
sorted(articles['tags'][0])

We now note that if this has worked we have sequences of LASTNAME, FIRSTNAME, ADDR, DATE in the tags.  We now run a process that extracts these sequences into name-address-date records in a new data frame. We keep track of the tags that are used to support the record as they will be useful later for training a new statistical model.

For each date we extract we try to parse it into day/month/year and add a field with a standardised date format.

In [ ]:
records = gazette.extract_records(articles)
records.head()

In [ ]:
records.support[0]

In [ ]:
records.shape

---
So we now have 81518 name address records. However, some of these will be invalid as our parse procedure is not
perfect.  We can remove some obvious errors by looking for things we don't expect:

- numbers in first or last name
- have a defaulted date field (couldn't parse date)
- address contains 'formerly' (probably contains more than one record)
- address is more than 70 chars (probably contains more than one record)

This is done by the procedure `valid_record` which we apply to every record to get a boolean vector, we can then use 
that to select just the good records. We get the bad ones too as we might be able to learn something from them.

In [ ]:
keep = records.apply(gazette.valid_record, axis=1)
good_records = records[keep]
bad_records = records[keep == False]

In [ ]:
good_records.shape

---
So we now have 70209 'good' records - those that look reasonable.  Let's take a look at a few of the good and bad ones.

In [ ]:
good_records.head()

In [ ]:
bad_records.head()

---
Write the good records out to a CSV file so that they could be passed to another process/application or just
examinied offline.  This cell will just write the main columns, ignoring the support column which isn't really
human readable.

In [ ]:
with open("good_records.csv", 'w') as fd:
    good_records.to_csv(fd, columns=['id', 'first', 'last', 'address', 'date', 'datestring'])